In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import os
from langchain_mcp_adapters.client import MultiServerMCPClient

github_pat = os.getenv("GITHUB_PAT")


# mcp_client = MultiServerMCPClient({
#     "github": {
#         "type": "http",
#         "url": "https://api.githubcopilot.com/mcp/",
#         "headers": {
#             "Authorization": f"Bearer {github_pat}"
#         },
#         "transport": "streamable_http",
#     },
# })


mcp_client = MultiServerMCPClient({
    "github" : {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_TOOLSETS",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server"
        ],
        "env": {
            "GITHUB_TOOLSETS": "context, pull_requests",
            "GITHUB_PERSONAL_ACCESS_TOKEN": github_pat
        },
        "transport": "stdio",
    }
})


In [ ]:
tool_list = await mcp_client.get_tools()

tool_list


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model = llm,
    tools = tool_list,
    state_modifier = ("Use the tools provided to you to answer the user`s questions.")
)

In [ ]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:
            key = list(chunk.keys())[0]
            
            if key == 'agent':
                content = chunk['agent']['messages'][0].content if chunk['agent']['messages'][0].content != '' else chunk['agent']['messages'][0].additional_kwargs
                print(f"'agent': '{content}'")
                
            elif key =='tools':
                for tool_msg in chunk['tools']['messages']:
                    print(f"'tool': '{tool_msg.content}'")
                    
            results.append(chunk)
        return results
    
    except Exception as e:
        print(f"Error processing streaming: {e}")
        return results

In [ ]:
from langchain_core.messages import HumanMessage

query = """ 깃헙의 Pull Request를 확인하고 코드 리뷰를 작성해주세요.

    PULL REQUEST URL: https://github.com/jasonkang14/sat-reading-client/pull/4

"""

stream_generator = agent.astream({'messages': [HumanMessage(content=query)]})


all_chunks = await process_stream(stream_generator)


if all_chunks: 
    final_result = all_chunks[-1]
    print(final_result)
